In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

In [ ]:
# df_ground_truth = pd.read_csv('../annotations/ground_truth_sample.csv', index_col=0, sep=';')
# df_yolo_predictions = pd.read_csv('../annotations/yolo_labels_sample.csv', index_col=0, sep=';')
# TODO: add +1 for frame_id in yolo real predictions

df_ground_truth = pd.read_csv('../annotations/ground_truth_boat_frames_coco.csv', index_col=0, sep=';')
df_yolo_predictions = pd.read_csv('../annotations/yolov8x_no_train_labels.csv', index_col=0, sep=';')

df_ground_truth['datetime'] = pd.to_datetime(df_ground_truth.datetime)
df_ground_truth['date'] = df_ground_truth.datetime.dt.date
df_yolo_predictions['datetime'] = pd.to_datetime(df_yolo_predictions['datetime'])
df_yolo_predictions['date'] = pd.to_datetime(df_yolo_predictions.datetime.dt.date, format='%Y-%m-%d')
print('shape of loaded data', df_ground_truth.shape, df_yolo_predictions.shape)
print(df_ground_truth.groupby(['date', 'camera_id']).datetime.count())
print(df_yolo_predictions.groupby(['date', 'camera_id']).datetime.count())

df_yolo_predictions = df_yolo_predictions[\
    ((df_yolo_predictions.date == '2023-06-09') & (df_yolo_predictions.camera_id.isin([1,2]))) |\
    ((df_yolo_predictions.date == '2023-06-10') & (df_yolo_predictions.camera_id == 2)) |\
    ((df_yolo_predictions.date == '2023-07-07') & (df_yolo_predictions.camera_id == 2)) |\
    ((df_yolo_predictions.date == '2023-07-08') & (df_yolo_predictions.camera_id == 1)) \
].copy()

df_yolo_predictions.drop(index=df_yolo_predictions[(df_yolo_predictions.camera_id == 2) & (df_yolo_predictions.x > 1800)].index, inplace=True)
df_ground_truth.drop(index=df_ground_truth[(df_ground_truth.camera_id == 2) & (df_ground_truth.x > 1800)].index, inplace=True)

df_ground_truth.set_index('filename', inplace=True)
df_yolo_predictions.set_index('filename', inplace=True)

df_ground_truth.shape, df_yolo_predictions.shape

In [ ]:
df_yolo_predictions_frame_indexed = df_yolo_predictions.reset_index().set_index(['filename', 'frame_id'])
df_yolo_predictions_frame_indexed.shape

In [ ]:
def calculate_iou(ground_truth:tuple, prediction:tuple):
    """
        Calculate intersection over union for two bounding boxes.
        Args:
            ground_truth: tuple of (x, y, w, h)
            prediction: tuple of (x, y, w, h)
    """
    gt_xtl = ground_truth[0]-ground_truth[2]/2
    gt_ytl = ground_truth[1]-ground_truth[3]/2
    gt_xbr = ground_truth[0]+ground_truth[2]/2
    gt_ybr = ground_truth[1]+ground_truth[3]/2
    pr_xtl = prediction[0]-prediction[2]/2
    pr_ytl = prediction[1]-prediction[3]/2
    pr_xbr = prediction[0]+prediction[2]/2
    pr_ybr = prediction[1]+prediction[3]/2
    intersection_xtl = max(gt_xtl, pr_xtl)
    intersection_ytl = max(gt_ytl, pr_ytl)
    intersection_xbr = min(gt_xbr, pr_xbr)
    intersection_ybr = min(gt_ybr, pr_ybr)
    intersection_area = max(0, intersection_xbr - intersection_xtl) * max(0, intersection_ybr - intersection_ytl)
    union_area = ground_truth[2] * ground_truth[3] + prediction[2] * prediction[3] - intersection_area
    return intersection_area / union_area

# group quality results by name, aggregate over frame_id and calculate true positive, false positive, false negative when comparing corresponding names and frame from both dataset
evaluation_dict = dict()
#for id in tqdm.tqdm(list(['cfg_raw_cam_02_fhd_h265_20230707T124001.mkv'])):
for id in tqdm.tqdm(set(df_ground_truth.index) | set(df_yolo_predictions.index)):
# for id in set(df_ground_truth.index) | set(df_yolo_predictions.index):
    evaluation_dict[id] = dict()

    if id in df_ground_truth.index:
        if df_ground_truth.loc[id,'frame_id'].size == 1:
            ground_truth_frame_ids = set([df_ground_truth.loc[id,'frame_id']])
        else:
            ground_truth_frame_ids = set(df_ground_truth.loc[id,'frame_id'])
    else:
        ground_truth_frame_ids = set()

    if id in df_yolo_predictions.index:
        if df_yolo_predictions.loc[id,'frame_id'].size == 1:
            yolo_frame_ids = set([df_yolo_predictions.loc[id,'frame_id']])
        else:
            yolo_frame_ids = set(df_yolo_predictions.loc[id,'frame_id'])
    else:
        yolo_frame_ids = set()
    
    corresponding_frames = ground_truth_frame_ids & yolo_frame_ids
    evaluation_dict[id]['true_positive'] = len(corresponding_frames)
    evaluation_dict[id]['false_positive'] = len(yolo_frame_ids - ground_truth_frame_ids)
    evaluation_dict[id]['false_negative'] = len(ground_truth_frame_ids - yolo_frame_ids)
    if len(corresponding_frames) > 0:
        frames_iou = {}
        for frame_id in corresponding_frames:
            # calulate iou for each frame
            ground_truth_frame = df_ground_truth.loc[id].loc[df_ground_truth.loc[id].frame_id == frame_id].iloc[0]
            # prediction_frame = df_yolo_predictions.loc[id].loc[df_yolo_predictions.loc[id].frame_id == frame_id].sort_values('w', ascending=False).iloc[0] ## this line was computation heavy, therefore indexed version is used
            prediciton_frames = df_yolo_predictions_frame_indexed.loc[id].loc[frame_id]
            if len(prediciton_frames.shape) == 1:
                prediction_frame = prediciton_frames
            else:
                prediction_frame = prediciton_frames.sort_values('w', ascending=False).iloc[0]
            frames_iou[frame_id] = calculate_iou(ground_truth_frame[['x', 'y', 'w', 'h']].values, prediction_frame[['x', 'y', 'w', 'h']].values)
        evaluation_dict[id]['iou'] = sum(frames_iou.values()) / len(frames_iou.values())
        evaluation_dict[id]['frames_iou'] = frames_iou
    else:
        evaluation_dict[id]['iou'] = 0
        evaluation_dict[id]['frames_iou'] = []

df_evaluation = pd.DataFrame().from_dict(evaluation_dict, orient='index')
df_evaluation['f1'] = 2 * df_evaluation['true_positive'] / (2 * df_evaluation['true_positive'] + df_evaluation['false_positive'] + df_evaluation['false_negative'])
df_evaluation['recall'] = df_evaluation['true_positive'] / (df_evaluation['true_positive'] + df_evaluation['false_negative'])
df_evaluation['precision'] = df_evaluation['true_positive'] / (df_evaluation['true_positive'] + df_evaluation['false_positive'])
df_evaluation

In [ ]:
df_evaluation.sort_values('true_positive', ascending=False)

In [ ]:
df_evaluation.to_csv('data_evaluation.csv')

## Helpers for debugging

In [ ]:
plt.scatter(df_evaluation.iloc[0].frames_iou.keys(), df_evaluation.iloc[0].frames_iou.values())

In [ ]:
df_ground_truth.loc['cfg_raw_cam_01_fhd_h265_20230609T050002.mkv'].sort_values('frame_id')

In [ ]:
df_ground_truth.loc['cfg_raw_cam_02_fhd_h265_20230707T124001.mkv'].sort_values('frame_id').frame_id.plot()

In [ ]:
convert_frame_id = 1087
convert_frame_id / 4 / 60, convert_frame_id / 4 // 60, convert_frame_id / 4 % 60

In [ ]:
df_tmp = df_yolo_predictions.loc['cfg_raw_cam_02_fhd_h265_20230707T124001.mkv']
df_tmp[df_tmp.x < 1800].sort_values('frame_id').frame_id.plot()

In [ ]:
df_tmp[(df_tmp.x < 1800) & (df_tmp.frame_id > 1000)].sort_values('frame_id')